In [1]:
import calendar
import pandas as pd
import numpy as np

In [2]:
def load_nysm_data(year):
    path_to_data= '/home/aevans/nwp_bias/data/nysm/'

    nysm_1H_obs = pd.read_parquet(f'{path_to_data}nysm_1H_obs_{year}.parquet')
    nysm_3H_obs = pd.read_parquet(f'{path_to_data}nysm_3H_obs_{year}.parquet')
    return nysm_1H_obs, nysm_3H_obs

In [3]:
def format_ok(df):
    #(subset=['tair','td','relh','wspd_sonic','wmax_sonic','wdir_sonic', 'precip_total','pres'])
    df = df.rename(columns={'STID': 'station', 'TIME': 'valid_time', 'PRES': 'pres', 'TAIR': 'tair', 'TDEW': 'td', 'RELH': 'relh', 'WSPD': 'wspd_sonic', 'WMAX': 'wmax_sonic', 'WDIR': 'wdir_sonic', 'RAIN': 'precip_total'})
    return df

In [4]:
def load_oksm_data(year):
    path_to_data= '/home/aevans/nwp_bias/data/oksm/'

    df_1H = pd.read_csv(f'{path_to_data}oksm_1H_obs_{year}.csv')
    df_3H = pd.read_csv(f'{path_to_data}oksm_3H_obs_{year}.csv')
    oksm_1H_obs = format_ok(df_1H)
    oksm_3H_obs = format_ok(df_3H)
    return oksm_1H_obs, oksm_3H_obs

In [5]:
def get_errors(model_data, nysm_obs, model, obs):
    model_error = (model_data[model] - nysm_obs[obs]).dropna()
    model_error = model_error.to_frame().rename(columns={0:f'{model}_error'})
    return model_error

def get_rmse(model_data, nysm_obs, model, obs):
    rmse = ((model_data[model] - nysm_obs[obs]) ** 2).mean() ** .5
    model_data[f'{model}_rmse'] = rmse
    return model_data[f'{model}_rmse']

In [6]:
def build_fcast_and_error_metrics_df(model, model_data, nysm_obs):
    if model=='HRRR':
        pres = 'mslma'
    else:
        pres = 'prmsl'
        
    model_data = model_data.copy()
    if pres in model_data.keys():
        model_data.loc[:,pres] = model_data[pres].div(100.) #convert Pa to hPa
    model_data_subset = model_data.reset_index().set_index(['station','valid_time','time'])[
        ['t2m','d2m','u_total','u_dir','latitude','longitude','new_tp',pres,'orog']]
    model_data_subset = model_data_subset.sort_values(by=['station','valid_time','time'])  
    nysm_obs_nonan = nysm_obs.dropna(subset=['tair','td','relh','wspd_sonic','wmax_sonic','wdir_sonic', 'precip_total','pres']).rename_axis(index={nysm_obs.index.names[1]:'valid_time'})

    # compute the error    
    model_obs_var_pairs = {'t2m':'tair', 'd2m':'td', 'u_total':'wspd_sonic',
                           'u_dir':'wdir_sonic', 'new_tp':'precip_total', pres:'pres'}
    model_error_save = []

    #print("model_data_subset", model_data_subset)
    #print("nysm_obs_nonan", nysm_obs_nonan)
    for key, value in model_obs_var_pairs.items():
        model_error_save.append(get_errors(model_data_subset, nysm_obs_nonan, key, value))
    model_error = pd.concat(model_error_save, axis=1, sort=False)

    
    model_fcast_and_error_df = pd.concat([model_data_subset, model_error], axis=1, sort=False).rename(
                                           columns={'time':'init_time','t2m':f't2m_{model}', 
                                                    'd2m':f'd2m_{model}',
                                                    'new_tp':f'new_tp_{model}',
                                                    'u_total':f'u_total_{model}',
                                                    'u_dir':f'u_dir_{model}',
                                                     pres:f'{pres}_{model}'}).dropna()
    # put the observations into the dataframe
    for key, value in model_obs_var_pairs.items():
        model_fcast_and_error_df[f'{key}_nysm'] = model_fcast_and_error_df[f'{key}_{model}']-model_fcast_and_error_df[f'{key}_error']
    
    return model_fcast_and_error_df

def build_fcast_and_error_metrics_df_ok(model, model_data, nysm_obs):
    if model=='HRRR':
        pres = 'mslma'
    else:
        pres = 'prmsl'
        
    model_data = model_data.copy()
    if pres in model_data.keys():
        model_data.loc[:,pres] = model_data[pres].div(100.) #convert Pa to hPa
    model_data_subset = model_data.reset_index().set_index(['station','valid_time','time'])[
        ['t2m','d2m','u_total','u_dir','latitude','longitude','new_tp',pres,'orog']]
    model_data_subset = model_data_subset.sort_values(by=['station','valid_time'])  
    nysm_obs_nonan = nysm_obs.dropna(subset=['tair','td','relh','wspd_sonic','wmax_sonic','wdir_sonic', 'precip_total','pres']).rename_axis(index={nysm_obs.index.names[0]:'valid_time'})

    # compute the error    
    model_obs_var_pairs = {'t2m':'tair', 'd2m':'td', 'u_total':'wspd_sonic', 'u_dir':'wdir_sonic', 'new_tp':'precip_total', pres:'pres'}
    
    model_error_save = []

    for key, value in model_obs_var_pairs.items():
        model_error_save.append(get_errors(model_data_subset, nysm_obs_nonan, key, value))
    model_error = pd.concat(model_error_save, axis=1, sort=False)
    
    model_fcast_and_error_df = pd.concat([model_data_subset, model_error], axis=1, sort=False).rename(columns={'time':'init_time','t2m':f't2m_{model}','d2m':f'd2m_{model}','new_tp':f'new_tp_{model}',
                                                    'u_total':f'u_total_{model}',
                                                    'u_dir':f'u_dir_{model}', pres:f'{pres}_{model}'}).dropna()
    # put the observations into the dataframe
    for key, value in model_obs_var_pairs.items():
        model_fcast_and_error_df[f'{key}_nysm'] = model_fcast_and_error_df[f'{key}_{model}']-model_fcast_and_error_df[f'{key}_error']
    
    return model_fcast_and_error_df

def add_in_fcast_times(model_fcast_and_error_df):
    # break down into days & hours of lead time
    model_fcast_and_error_df = model_fcast_and_error_df.reset_index()
    lead_time_delta = model_fcast_and_error_df['valid_time'] - \
                                          model_fcast_and_error_df['time']

    model_fcast_and_error_df['lead_time_DAY'] = lead_time_delta.dt.days
    model_fcast_and_error_df['lead_time_HOUR'] = divmod(lead_time_delta.dt.seconds, 3600)[0]
    model_fcast_and_error_df['lead_time_ONLY_HOURS'] = (24. * model_fcast_and_error_df['lead_time_DAY']) + \
                                                        model_fcast_and_error_df['lead_time_HOUR']
    return model_fcast_and_error_df

In [7]:
def main(model, init, month, year, state, mask_water=True):
    '''
    This function matches observation and forecast times to calculate error (forecast - observation) for all specified variables.
    
    The following parameters need to be passed into main():
    
    model (str) - HRRR, NAM, GFS
    init(str) - initilization time for model, '00' or '12' UTC
    month (str) - integer corresponding to calendar month (e.g. '01' is January, '02' is Februrary, etc.)
    year (str) - the year of interest (e.g., '2020')
    mask_water (bool) - true to mask out grid cells over water before interpolation/nearest-neighbor, 
                        false to leave all grid cells available for interpolation/nearest-neighbor
    '''

    if state == 'NY': 
        path_to_data_ny= f'/home/aevans/ai2es/processed_data/{model}/ny/'
        
        #ny
        if mask_water:
            print('using data over land only')
            df_model_nysm_sites = pd.read_parquet(f'{path_to_data_ny}{model}_{init}z_{month}-{year}_interp_to_nysm_sites_mask_water.parquet')
        else:
            df_model_nysm_sites = pd.read_parquet(f'{path_to_data_ny}{model}_{init}z_{month}-{year}_interp_to_nysm_sites.parquet')
        df_model_nysm_sites = df_model_nysm_sites.reset_index().set_index(['station','valid_time'])
        model_list_ny = [df_model_nysm_sites]

        nysm_1H_obs, nysm_3H_obs = load_nysm_data(year)

        if model=='GFS':
            nysm_obs_list = [nysm_3H_obs]
        elif model=='HRRR':
            nysm_obs_list = [nysm_1H_obs]
        elif model=='NAM':
            nysm_obs_list = [nysm_1H_obs, nysm_3H_obs]
        
        #NY
        df_model_nysm_sites_le_36 = df_model_nysm_sites[df_model_nysm_sites['lead time'] <= 36.]
        df_model_nysm_sites_gt_36 = df_model_nysm_sites[df_model_nysm_sites['lead time'] > 36.]
        model_list_ny = [df_model_nysm_sites_le_36, df_model_nysm_sites_gt_36]

        df_save_ny = []
        # NY
        for nysm_obs, model_df in zip(nysm_obs_list, model_list_ny):
            model_fcast_and_error_df = build_fcast_and_error_metrics_df(model, model_df, nysm_obs)
            model_fcast_and_error_df = add_in_fcast_times(model_fcast_and_error_df)
            model_fcast_and_error_df = model_fcast_and_error_df.set_index(['station', 'valid_time', 'time'])
            df_save_ny.append(model_fcast_and_error_df)
        model_fcast_and_error_df = pd.concat(df_save_ny, sort=False)


    if state == 'OK':
        path_to_data_ok= f'/home/aevans/ai2es/processed_data/{model}/ok/'

        #OK
        if mask_water:
            print('using data over land only')
            df_model_oksm_sites = pd.read_parquet(f'{path_to_data_ok}{model}_{init}z_{month}-{year}_interp_to_oksm_sites_mask_water.parquet')
        else:
            df_model_oksm_sites = pd.read_parquet(f'{path_to_data_ok}{model}_{init}z_{month}-{year}_interp_to_oksm_sites.parquet')
        df_model_oksm_sites = df_model_oksm_sites.reset_index().set_index(['station','valid_time'])
        model_list_ok = [df_model_oksm_sites]
        
        oksm_1H_obs, oksm_3H_obs = load_oksm_data(year)

        if model=='GFS':
            oksm_obs_list = [oksm_3H_obs] 
        elif model=='HRRR':
            oksm_obs_list = [oksm_1H_obs]
        elif model=='NAM':
            oksm_obs_list = [oksm_1H_obs, oksm_3H_obs]
            # need to make sure I'm computing error with appropriate 1H and 3H observations, so need to split up
            # the model data to do so dynamically

            #OK
            df_model_oksm_sites_le_36 = df_model_oksm_sites[df_model_oksm_sites['lead time'] <= 36.]
            df_model_oksm_sites_gt_36 = df_model_oksm_sites[df_model_oksm_sites['lead time'] > 36.]
            model_list_ok = [df_model_oksm_sites_le_36, df_model_oksm_sites_gt_36]
        
        df_save_ok = []
        #OK
        for oksm_obs, model_df_ok in zip(oksm_obs_list, model_list_ok):
            model_fcast_and_error_df = build_fcast_and_error_metrics_df_ok(model, model_df_ok, oksm_obs)
            model_fcast_and_error_df = add_in_fcast_times(model_fcast_and_error_df)
            model_fcast_and_error_df = model_fcast_and_error_df.set_index(['station', 'valid_time', 'time'])
            df_save_ok.append(model_fcast_and_error_df)
        model_fcast_and_error_df = pd.concat(df_save_ok, sort=False)
        
    return model_fcast_and_error_df

In [8]:
# Observations, forecasts, and forecast error calculated within these main function calls are all saved to the same parquet file.
for year in np.arange(2018,2022):
    init = '12' # enter init time here
    nam_fcast_and_error_df_save = []
    gfs_fcast_and_error_df_save = []
    hrrr_fcast_and_error_df_save = []

    print(year)
    
    for month in range(1, 13):
        print(calendar.month_name[month])
        print('NAM')
        nam_fcast_and_error_df_save.append(main('NAM', init, str(month).zfill(2), year, 'NY'))
        print('HRRR')
        hrrr_fcast_and_error_df_save.append(main('HRRR', init, str(month).zfill(2), year, 'NY'))
        print('GFS')
        gfs_fcast_and_error_df_save.append(main('GFS', init, str(month).zfill(2), year, 'NY'))

    nam_fcast_and_error_df = pd.concat(nam_fcast_and_error_df_save, sort=False)
    gfs_fcast_and_error_df = pd.concat(gfs_fcast_and_error_df_save)
    hrrr_fcast_and_error_df = pd.concat(hrrr_fcast_and_error_df_save)

    savedir = '/home/aevans/ai2es/processed_data/frcst_err/'
    nam_fcast_and_error_df.to_parquet(f'{savedir}nam_fcast_and_error_df_{init}z_{year}_mask_water_ny.parquet')
    gfs_fcast_and_error_df.to_parquet(f'{savedir}gfs_fcast_and_error_df_{init}z_{year}_mask_water_ny.parquet')
    hrrr_fcast_and_error_df.to_parquet(f'{savedir}hrrr_fcast_and_error_df_{init}z_{year}_mask_water_ny.parquet')

2018
January
NAM
using data over land only


KeyboardInterrupt: 